In [1]:
# importing libraries
import numpy as np
import pandas as pd

# importing library to search text in a document
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

#import library to convert text to feature vector
from sklearn.feature_extraction.text import TfidfVectorizer

# import models
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SOUMITA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#printing stopwordsin english
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

Data collecting and Data preprocessing

In [4]:
news= pd.read_csv("fake news train.csv")
news.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
# counting missing value
news.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
# replacing null values with empty string
news= news.fillna(' ')

In [7]:
# combining author and title name
news['content'] = news['author']+': '+ news['title']
news

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus: House Dem Aide: We Didn’t Even ...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn: FLYNN: Hillary Clinton, Big W..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com: Why the Truth Might Get Yo...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss: 15 Civilians Killed In Single...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy: Iranian woman jailed for ficti...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Jerome Hudson: Rapper T.I.: Trump a ’Poster Ch...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"Benjamin Hoffman: N.F.L. Playoffs: Schedule, M..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Michael J. de la Merced and Rachel Abrams: Mac...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex Ansary: NATO, Russia To Hold Parallel Exe..."


In [8]:
# Now preprocessing a text to number
port_stem= PorterStemmer()
def stemming(content):
    stemmed_content= re.sub('[^a-zA-Z]',' ', content)
    # converting uppercase word to in lower case
    stemmed_content= stemmed_content.lower()
    
    # Now splitting all the words in list and reducing it into root word also removing stopwords
    stemmed_content= stemmed_content.split()
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    
    # now joining all  the words
    stemmed_content=' '.join(stemmed_content)
    return stemmed_content

In [9]:
news["content"]=news["content"].apply(stemming)
print(news['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [10]:
# separating the data and labels
X= news['content'].values
Y= news['label'].values

Now through vectorizer function we will convert text data to numerical data

In [11]:
Vectorizer= TfidfVectorizer()
Vectorizer.fit(X)

TfidfVectorizer()

In [12]:
X= Vectorizer.transform(X)
X

<20800x17128 sparse matrix of type '<class 'numpy.float64'>'
	with 210687 stored elements in Compressed Sparse Row format>

In [13]:
X_train,X_test, Y_train, Y_test= train_test_split(X,Y, test_size=0.2, stratify= Y, random_state=2)
# need to use stratify to get similar proportion of data in train as in file

Training the model

In [14]:
Lg= LogisticRegression()

In [15]:
Lg.fit(X_train, Y_train)

LogisticRegression()

In [16]:
X_predict=Lg.predict(X_train)
accuracy_score(X_predict, Y_train)

0.9865985576923076

In [17]:
X_test_predict=Lg.predict(X_test)
accuracy_score(X_test_predict, Y_test)

0.9790865384615385

Make a predictive System

In [18]:
X_new= X_test[3]

prediction= Lg.predict(X_new)
print(prediction)

if prediction[0]==0:
    print("The news is real")
else:
    print("The news is fake")

[0]
The news is real
